# Introducción

En este cuaderno se va a presentar el apartado en el que el proyecto comienza a utilizar las librerías python de pandas y también el uso de Scikit-Learn:
* 1.- Cargar los archivos .csv de las pruebas recogidas, tanto benignas como malignas.
* 2.- Realizar la unión de ambos archivos en uno solo, añadiendo una columna final con valores "0" para tráfico benigno y "1" para tráfico maligno
* 3.- Eliminar columnas innecesarias que producen ruido
* 4.- Completar el archivo final para el estudio realizando la normalización usando MinMaxScaler()

# Normalización del documento

## Cargar archivos

En este primer paso, se procede a cargar los archivos generados tras la ejecución de Dorothea tanto con la opción "-t attack" como con "-t normal"
Se generan una serie de interacciones en el sistemas que permiten generar archivos .csv con todo el flujo de datos de red necesario para el estudio.
Tras generar dichos .csv, estos se cargan desde el script gestionarchivos.py de la siguiente manera:

In [ ]:
# Ruta al archivo CSV
maligno = "/home/osboxes/Documents/Dorothea/Labs/lab_attacks/results/Pruebas/Maligno4.csv"
benigno = "/home/osboxes/Documents/Dorothea/Labs/lab_normal/results/benigno1.csv"

## Modificación y unión de archivos

A continuación se agregan los valores "0" para tráfico benigno y "1" para tráfico maligno en los archivos .csv anteriormente indicados. Para ello, en primer lugar se crean dos variables con dichos valores para posteriormente agregarlos a los archivos .csv cargados en una nueva columna denominada "Tipo Trafico". Se finaliza este paso unificando ambos archivos:

In [ ]:
# Valor fijo a añadir en la nueva columna
valor_maligno = 1
valor_benigno = 0

# Cargar archivos CSV
df1 = pd.read_csv(maligno)
df2 = pd.read_csv(benigno)

# Agregar una columna con un valor fijo
df1['Tipo Trafico'] = valor_maligno
df2['Tipo Trafico'] = valor_benigno

# Se unen los archivos en uno
df_concatenado = pd.concat([df1, df2], axis=0)

## Eliminación de columnas innecesarias

Para una mejor gestión de los datos y una mejor comprensión, se proceden a eliminar las columnas que introducen ruido en las muestras

In [ ]:
# Se eliminan columnas innecesarias
columnas_eliminar = ['engine_type', 'engine_id', 'tos', 'src_mask', 'dst_mask', 'src_as', 'dst_as' ]  
df_concatenado = df_concatenado.drop(columnas_eliminar, axis=1)

## Gestión de las columnas que contienen direcciones IP

Para normalizar las columnas del archivo .csv que tienen formato de dirección IP, en primer lugar se transforman dichas columnas a un formato de Bytes para posteriormente pasarlas a enteros.
Para ello se define la función normalize_ip

In [ ]:
# Se define la función que realizará la conversión de las IPs a enteros
def normalize_ip(ip):
    try:
        # Convierte la dirección IP en formato de bytes
        packed_ip = socket.inet_aton(ip)
        packed_ip = int.from_bytes(packed_ip, byteorder='big')
        return packed_ip
    except socket.error:
        # Si la dirección IP es inválida, devuelve None
        return None

# Se definen las columnas del archivo que tienen las direcciones IP que se tienen que normalizar
DireccionesIP = ["exaddr", "srcaddr", "dstaddr", "nexthop"]
for direccion in DireccionesIP:
    df_concatenado[direccion] = df_concatenado[direccion].apply(normalize_ip)


## Archivo normalizado

Para finalizar este primer apartado, se procede con la normalización del documento a través de la función MinMaxScaler() facilitada por la libreria Scikit-Learn.
De esta manera, se realizará el entrenamiento de nuestra inteligencia con valores entre 0 y 1, permitiendo así, que las pruebas realizadas no sufran errores.

In [ ]:
# Se realiza ahora la normalización se las columnas para 
# posteriormente trabajar con ellas

columnas = ['#:unix_secs', 'unix_nsecs', "exaddr", 'sysuptime', 'dpkts', 'doctets', 'first', 'last', "srcaddr", "dstaddr", "nexthop", "input", 'output', 'srcport', 'dstport', 'prot', 'tcp_flags', 'Tipo Trafico']
scaler = MinMaxScaler()
df_concatenado[columnas] = scaler.fit_transform(df_concatenado[columnas])
df_concatenado.to_csv('csv_normalizado.csv', index=False)
